# Test the functions in Project 1

This is one of three mandatory projects of the course FYS-STK4155 (University of Oslo). In this file, we test the performance of our own functions by comparing them with the functions in the scikit learn python package.

The Frank2 function was given for this project, so we are not going to test it.

In [1]:
def franke_function(x, y):
    term1 = 0.75 * np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75 * np.exp(-((9*x+1)**2) / 49.0 - 0.1*(9*y+1))
    term3 = 0.5 * np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2 * np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4

Import packages

In [2]:
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.linear_model import LinearRegression 
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error, mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import algorithms
from model_selection import GridSearch #use Gridsearch.fit(x,y)
from model_comparison import model_comparison
from functions import generateDesignmatrix

#  1) generateDesignmatrix in functions.py

This functions generates a design matrix for the arrays x and y, and a given polynonial order p. The arrays x and y are the ones that define the meshgrid for the calculation of z with the Franke function.

In [3]:

""""
def generateDesignmatrix(p, x, y): 
    m = int((p**2+3*p+2)/2) # returnerer heltall for p = [1:5]
    X = np.zeros((len(x), m))
    X[:,0] = 1
    counter = 1
    for i in range(1, p+1):
        for j in range(i+1):
            X[:,counter] = x**(i-j) * y**j
            counter+=1
    return X
    """

'"\ndef generateDesignmatrix(p, x, y): \n    m = int((p**2+3*p+2)/2) # returnerer heltall for p = [1:5]\n    X = np.zeros((len(x), m))\n    X[:,0] = 1\n    counter = 1\n    for i in range(1, p+1):\n        for j in range(i+1):\n            X[:,counter] = x**(i-j) * y**j\n            counter+=1\n    return X\n    '

In [4]:
np.random.seed(1000)
# We work only with squared matrices. This implies that the length of x and y is the same.
m = 5  # m defines the size of the meshgrid.
x = np.random.rand(m, )
y = np.random.rand(m, )
z = franke_function(x, y)

# x, y, and z are arrays. The length of x and y is m, which determines the dimensions of z (mxm).

In [5]:
print('array x:', x)
print('array y:', y)
print('array z:', z)

array x: [0.65358959 0.11500694 0.95028286 0.4821914  0.87247454]
array y: [0.21233268 0.04070962 0.39719446 0.2331322  0.84174072]
array z: [0.50251014 0.90614694 0.32573166 0.56740664 0.0665042 ]


In [6]:
# Test the designMatrix for p=0. 
# When p=0 we expect the designMatrix functions to generate a matrix X which only consist of 
# the terms that correspond to the intercept.
# This means that X has only a column of length equal to the length of x and y, and all its elements are 1.
p = 0 # [1:5]
X = generateDesignmatrix(p,x,y)
print('Design matrix:', X) 

Design matrix: [[1.]
 [1.]
 [1.]
 [1.]
 [1.]]


In [7]:
# Test the designMatrix for p=0. 
# When p=0 we expect the designMatrix functions to generate a matrix X with three columns, the first filled with ones, 
# as we have seen before, and the second and third with the values of x and y.
p = 1 # [1:5]
X = generateDesignmatrix(p,x,y)
print('Design matrix:', X) 

Design matrix: [[1.         0.65358959 0.21233268]
 [1.         0.11500694 0.04070962]
 [1.         0.95028286 0.39719446]
 [1.         0.4821914  0.2331322 ]
 [1.         0.87247454 0.84174072]]


In [8]:
# More general, the columns of the design matrix will be:
# ones, x, y, x^2, y^2, xy,..., x^py^0,..., x^(p-i)y^i, ..., x^0y^p.
# Thus, for a given p, we have t = (p^2+3p+2)/2 number of columns in the design matrix X.

In [9]:
# Now, let's try with a higher order poynomial.
p = 3

X = generateDesignmatrix(p,x,y)
print('Design matrix:', X) 


Design matrix: [[1.00000000e+00 6.53589585e-01 2.12332681e-01 4.27179346e-01
  1.38778429e-01 4.50851674e-02 2.79199972e-01 9.07041358e-02
  2.94671959e-02 9.57305446e-03]
 [1.00000000e+00 1.15006943e-01 4.07096248e-02 1.32265970e-02
  4.68188950e-03 1.65727355e-03 1.52115049e-03 5.38449799e-04
  1.90597965e-04 6.74669843e-05]
 [1.00000000e+00 9.50282864e-01 3.97194461e-01 9.03037522e-01
  3.77447090e-01 1.57763440e-01 8.58141083e-01 3.58681502e-01
  1.49919894e-01 6.26627646e-02]
 [1.00000000e+00 4.82191401e-01 2.33132197e-01 2.32508548e-01
  1.12414341e-01 5.43506214e-02 1.12113622e-01 5.42052286e-02
  2.62074023e-02 1.26708798e-02]
 [1.00000000e+00 8.72474535e-01 8.41740724e-01 7.61211815e-01
  7.34397347e-01 7.08527447e-01 6.64137924e-01 6.40742984e-01
  6.18172155e-01 5.96396406e-01]]


In [10]:
# As expected, X has 10 columns. t = (3^3 + 3*3 + 2)/2 = 10
# The first column consists of ones, the second and the third are the vectors x and y.

# Let's check wether the column number 9 is equal to x*y^2

y2 = y**2
col9 = X[:, 8]

print('Has the ninth column in X the values we expect:', col9==x*y2)

Has the ninth column in X the values we expect: [ True  True  True  True  True]


# 2) OLS

We test our own code for the OLS comparing it with the scikit-learn function
When running OLS, x cannot be equal to y since that generates a singular design matrix, and our code will not run.

In [11]:
x = np.linspace(1, 10, num=10)
y = np.linspace(1.1, 10.1, num=10)
z = franke_function(x, y)
p = 2
X = generateDesignmatrix(p,x,y)

In [12]:
our_ols = algorithms.OLS()
our_ols.fit(X,z)
our_betas = our_ols.beta
our_betas

array([ 0.02929223, -0.01249085,  0.00209199, -0.00138651,  0.00031428,
        0.00180701])

In [13]:
# Create linear regression object
scikit_ols = LinearRegression(fit_intercept=False)

# Train the model using the training sets
scikit_ols.fit(X, z)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

In [14]:
# get the coefficients (?)
scikit_ols.coef_[0:9]

array([ 0.02986962, -0.00647389, -0.00348693,  0.00079619,  0.0001488 ,
       -0.00019989])

In the example above, all the coefficients are close to zero, both with our function and with the scikit.learn function. Nevertheless, we notice that the signs of some of the coefficients are different. The intercept is the same.

Now, we try with random selection of x and y and some larger samples

In [24]:
np.random.seed(1000)
m = 30  # m defines the size of the meshgrid.
x = np.random.rand(m, )
y = np.random.rand(m, )
z = franke_function(x, y)
p = 2
X = generateDesignmatrix(p,x,y)

In [25]:
our_ols = algorithms.OLS()
our_ols.fit(X,z)
our_betas = our_ols.beta
our_betas

array([ 1.15707568, -1.22131811, -0.72666861,  0.24054396,  1.13194101,
       -0.43215975])

In [26]:
X@our_betas

array([0.13468539, 0.12713231, 0.21728842, 0.6082088 , 0.31257862,
       0.83608651, 0.4222509 , 0.53106781, 0.13968897, 0.32624696,
       0.62073605, 0.38480847, 0.31299295, 0.60272371, 0.38959959,
       0.15241299, 0.15293553, 0.23529875, 0.24239559, 0.49563065,
       0.54789871, 0.26750687, 0.33570208, 0.41054777, 0.46519288,
       0.587262  , 0.3109471 , 0.45247379, 0.37315466, 0.53325455])

In [30]:
# Create linear regression object
scikit_ols = LinearRegression(fit_intercept=False)

# Train the model using the training sets
scikit_ols.fit(X, z)

#LinearRegression(fit_intercept=True)
# get the coefficients
scikit_ols.coef_[:]

array([ 1.15707568, -1.22131811, -0.72666861,  0.24054396,  1.13194101,
       -0.43215975])

In this case, we got the same coefficients.

# 3) Ridge

In [28]:
our_ridge = algorithms.Ridge(lmd = 0.1)
our_ridge.fit(X,z)
our_betas = our_ridge.beta
our_betas # Why so many betas????????

array([[  2.58637884,  -5.20102604,  -4.92214334,   2.75780707,
          3.6580606 ,   2.3627759 ],
       [ -5.42102216,  14.80267509,   7.78246852,  -9.87370121,
         -8.02950182,  -2.47303224],
       [ -5.1724349 ,   7.90849429,  13.6296366 ,  -3.23270898,
         -7.1013668 ,  -8.97466992],
       [  2.96529783, -10.04697725,  -3.31098467,   7.76110688,
          4.04332324,   0.63963639],
       [  3.84633705,  -8.01301441,  -6.97475391,   3.87948043,
          7.76531836,   2.33868419],
       [  2.61939318,  -2.82606428,  -9.2293133 ,   0.77048081,
          2.66004703,   7.75944197]])

In [29]:
# X@our_betas

array([[ 0.00894674,  0.33240151, -0.45144746, -0.39358692,  0.42790914,
         0.50781533],
       [-0.13323609,  0.37555856, -0.01795166, -0.16474398, -0.65065617,
         0.55905532],
       [ 0.09712095, -0.20433938, -0.44457359,  0.38838277, -0.31777987,
         0.52078416],
       [ 0.42906443, -0.11346204, -1.2252047 , -0.2959507 ,  0.49091075,
         0.77710946],
       [-0.17374603,  0.12134076,  0.64075395,  0.13440184, -0.19029191,
        -0.53994579],
       [ 1.0349169 , -1.73369176, -1.96347834,  0.70377304,  1.45584935,
         0.87422515],
       [ 0.14582136, -0.69977088,  0.20789435,  0.59805155, -0.10200091,
        -0.15442046],
       [-0.1349897 ,  0.52717235,  0.50741949, -0.49106387, -0.07112757,
        -0.49205506],
       [-0.24938481,  0.83424717,  0.11490358, -0.57750117, -0.59215234,
         0.42839117],
       [-0.14011661,  0.25845972,  0.38475058, -0.02314984, -0.25100556,
        -0.27960541],
       [ 0.18832946, -0.39238177,  0.09121615,  0.

In [35]:
from sklearn import linear_model
scikit_ridge=linear_model.RidgeCV(alphas=[0.1], fit_intercept=False)
scikit_ridge.fit(X, z)

#LinearRegression(fit_intercept=True)
# get the coefficients
scikit_ridge.coef_[:]

array([ 0.7771756 , -0.31589716, -0.16606758, -0.25684846,  0.4192761 ,
       -0.5259916 ])

The betas are clearly different!!! Here we have an error